In [ ]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
from datetime import datetime

In [ ]:
# Read & prep data
df = pd.read_csv('MTA_Daily_Ridership.csv')
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
# Create Dash app
app = dash.Dash(__name__)

# Define layout
app.layout = html.Div([
    html.H1('MTA Transit Ridership Dashboard', 
            style={'textAlign': 'center', 'color': '#2c3e50', 'marginBottom': 30}),
    
    html.Div([
        html.Div([
            html.Label('Select Transit Type:'),
            dcc.Dropdown(
                id='transit-type',
                options=[
                    {'label': 'Subway', 'value': 'Subways'},
                    {'label': 'Bus', 'value': 'Buses'},
                    {'label': 'LIRR', 'value': 'LIRR'},
                    {'label': 'Metro-North', 'value': 'Metro-North'},
                    {'label': 'Staten Island Railway', 'value': 'Staten Island Railway'}
                ],
                value='Subways',
                style={'width': '100%'}
            )
        ], style={'width': '30%', 'display': 'inline-block'}),
        
        html.Div([
            html.Label('Select Date Range:'),
            dcc.DatePickerRange(
                id='date-range',
                min_date_allowed=df['Date'].min(),
                max_date_allowed=df['Date'].max(),
                start_date=df['Date'].max() - pd.Timedelta(days=90),
                end_date=df['Date'].max()
            )
        ], style={'width': '40%', 'display': 'inline-block', 'marginLeft': '5%'})
    ], style={'marginBottom': 30}),
    
    dcc.Graph(id='ridership-graph'),
    
    dcc.Graph(id='recovery-graph')
])

@app.callback(
    [Output('ridership-graph', 'figure'),
     Output('recovery-graph', 'figure')],
    [Input('transit-type', 'value'),
     Input('date-range', 'start_date'),
     Input('date-range', 'end_date')]
)
def update_graphs(transit_type, start_date, end_date):
    # Filter data based on date range
    mask = (df['Date'] >= start_date) & (df['Date'] <= end_date)
    filtered_df = df.loc[mask]
    
    # Create ridership trend graph
    ridership_fig = px.line(
        filtered_df,
        x='Date',
        y=f'{transit_type}: Total Estimated Ridership',
        title=f'{transit_type} Daily Ridership Trend'
    )
    ridership_fig.update_layout(
        xaxis_title='Date',
        yaxis_title='Total Ridership',
        hovermode='x'
    )
    
    # Create recovery percentage graph
    recovery_fig = px.line(
        filtered_df,
        x='Date',
        y=f'{transit_type}: % of Comparable Pre-Pandemic Day',
        title=f'{transit_type} Recovery vs Pre-Pandemic Levels'
    )
    recovery_fig.update_layout(
        xaxis_title='Date',
        yaxis_title='Percentage of Pre-Pandemic Ridership',
        hovermode='x'
    )
    recovery_fig.add_hline(
        y=100,
        line_dash="dash",
        line_color="red",
        annotation_text="Pre-Pandemic Level"
    )
    
    return ridership_fig, recovery_fig

if __name__ == '__main__':
    app.run_server(debug=True)